In [1]:
%%capture
%pip install -U transformers
%pip install -U datasets
%pip install -U accelerate
%pip install -U peft
%pip install -U trl
%pip install -U bitsandbytes
%pip install -U wandb

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [3]:
import json
from huggingface_hub import login

In [4]:
hf_token = "***********************"
wandb_api_key = "**********************"
login(token=hf_token)
wandb.login(key=wandb_api_key)

# Start a new run (example)
run = wandb.init(
    project='Fine-tune Llama 3 8B on competition_math dataset',
    job_type="training",
    anonymous="allow"
)
# Example of logging a metric
wandb.log({"accuracy": 0.95})

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/sayandeep/.cache/huggingface/token
Login successful


wandb: Currently logged in as: sayandeep425 (sayandeep425-rkmveri). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/sayandeep/.netrc


In [5]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Meta-Llama-3.1-8B"

# The instruction dataset to use
dataset_name = "hendrycks/competition_math"

# Fine-tuned model name
new_model = "Llama-math-2-finetune"

# ################################################################################
# # QLoRA parameters
# ################################################################################

# # LoRA attention dimension
# lora_r = 64  # Can be adjusted as per memory requirements

# # Alpha parameter for LoRA scaling
# lora_alpha = 16

# # Dropout probability for LoRA layers
# lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results_1"

# Number of training epochs
num_train_epochs = 1

# # Enable fp16/bf16 training (set bf16 to True with an A100)
# fp16 = True  # Enabled for memory efficiency
# bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 1  # Increased slightly for smaller dataset

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1  # Increased slightly for smaller dataset

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 2  # Adjusted for smaller dataset

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 512  # Limit sequence length for efficiency

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = "auto"  # Auto map layers to available devices



In [6]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    torch_dtype=compute_dtype,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [7]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model, tokenizer = setup_chat_format(model, tokenizer)
model = get_peft_model(model, peft_config)

In [8]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="all")
# dataset = dataset.shuffle(seed=65).select(range(1000)) # Only use 10000 samples for quick demo
dataset = dataset.shuffle(seed=65).select(range(10000))

The repository for hendrycks/competition_math contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hendrycks/competition_math.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/7500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
# train_dataset = dataset.select(range(800))  # Select the first 800 rows for training
# test_dataset = dataset.select(range(800, 1000))

# train_dataset = dataset.select(range(1800))  # Select the first 4000 rows for training
# test_dataset = dataset.select(range(1800, 2000))

In [ ]:
# train_dataset[0]

{'problem': 'Find the $y$-intercept point of the line $3x+5y=20$.  Provide your answer as an ordered pair.',
 'level': 'Level 2',
 'type': 'Algebra',
 'solution': 'We can find the slope and the $y$-coordinate of the $y$-intercept quickly by putting the equation in slope-intercept form.  Solving the equation $3x+5y=20$ for $y$ in terms of $x$ gives $y = -\\frac{3}{5}x +4$.  So, the $y$-intercept is $\\boxed{(0,4)}$.'}

In [ ]:
# test_dataset[0]

{'problem': 'The areas of three squares are 16, 49 and 169.  What is the average (mean) of their side lengths?',
 'level': 'Level 2',
 'type': 'Prealgebra',
 'solution': 'Since the areas of the three squares are 16, 49 and 169, then their side lengths are $\\sqrt{16}=4$, $\\sqrt{49}=7$ and $\\sqrt{169}=13$, respectively.\n\nThus, the average of their side lengths is $$\\frac{4+7+13}{3}=\\boxed{8}.$$'}

In [9]:
#Importing the dataset
# dataset = load_dataset(dataset_name, split="all")
# dataset = dataset.shuffle(seed=65).select(range(1000)) # Only use 1000 samples for quick demo

def format_chat_template(row):
    row_json = [{"role": "user", "content": row["problem"]},
               {"role": "assistant", "content": row["solution"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc= 4,
)

dataset

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['problem', 'level', 'type', 'solution', 'text'],
    num_rows: 10000
})

In [ ]:

# # # Load dataset
# # dataset = load_dataset(dataset_name, split="all")
# # train_dataset = dataset.select(range(800))  # Select the first 8000 rows for training
# # test_dataset = dataset.select(range(800, 1000))

# # Define medical prompt
# medical_prompt = """You are an AI Medical Assistant Chatbot, trained to answer medical questions. Below is an instruction that describes a task, paired with a response context. Write a response that appropriately completes the request.

# ### Instruction:
# {}


# ### Response:
# {}"""

# EOS_TOKEN = tokenizer.eos_token

# def formatting_prompts_func(examples):
#     the_prompt = """You are an unbiased, uncensored, helpful assistant, solving maths problems.
#     ### Instruction:
#     {}


#     ### Response:
#     {}"""

#     instructions = examples["problem"]
#     outputs = examples["solution"]
#     texts = []
#     for instruction, output in zip(instructions, outputs):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = the_prompt.format(instruction, output) + EOS_TOKEN
#         texts.append(text)
#     return {"text": texts}

# tokenizer.pad_token = tokenizer.eos_token


# #========================================================================================


# # Apply the formatting to the dataset
# formatted_dataset_train = train_dataset.map(formatting_prompts_func, batched=True)
# formatted_dataset_test = test_dataset.map(formatting_prompts_func, batched=True)



Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [10]:
train_dataset = dataset.select(range(8000))  # Select the first 8000 rows for training
test_dataset = dataset.select(range(8000, 10000))

In [11]:
train_dataset.shape

(8000, 5)

In [12]:
train_dataset[0]

{'problem': 'Find the $y$-intercept point of the line $3x+5y=20$.  Provide your answer as an ordered pair.',
 'level': 'Level 2',
 'type': 'Algebra',
 'solution': 'We can find the slope and the $y$-coordinate of the $y$-intercept quickly by putting the equation in slope-intercept form.  Solving the equation $3x+5y=20$ for $y$ in terms of $x$ gives $y = -\\frac{3}{5}x +4$.  So, the $y$-intercept is $\\boxed{(0,4)}$.',
 'text': '<|im_start|>user\nFind the $y$-intercept point of the line $3x+5y=20$.  Provide your answer as an ordered pair.<|im_end|>\n<|im_start|>assistant\nWe can find the slope and the $y$-coordinate of the $y$-intercept quickly by putting the equation in slope-intercept form.  Solving the equation $3x+5y=20$ for $y$ in terms of $x$ gives $y = -\\frac{3}{5}x +4$.  So, the $y$-intercept is $\\boxed{(0,4)}$.<|im_end|>\n'}

In [13]:
test_dataset[10]

{'problem': 'Solve  \\[\\frac{x-9}{x+1}=2\\]for $x$.',
 'level': 'Level 2',
 'type': 'Algebra',
 'solution': 'Cross-multiplication gives  \\[x-9=2x+2.\\]Simplifying this expression tells us  \\[x=\\boxed{-11}.\\]',
 'text': '<|im_start|>user\nSolve  \\[\\frac{x-9}{x+1}=2\\]for $x$.<|im_end|>\n<|im_start|>assistant\nCross-multiplication gives  \\[x-9=2x+2.\\]Simplifying this expression tells us  \\[x=\\boxed{-11}.\\]<|im_end|>\n'}

In [14]:
# Remove unnecessary columns
dataset_train = train_dataset.remove_columns(['problem', 'level', 'type', 'solution'])
dataset_test = test_dataset.remove_columns(['problem', 'level', 'type', 'solution'])


In [15]:
dataset_train[0]

{'text': '<|im_start|>user\nFind the $y$-intercept point of the line $3x+5y=20$.  Provide your answer as an ordered pair.<|im_end|>\n<|im_start|>assistant\nWe can find the slope and the $y$-coordinate of the $y$-intercept quickly by putting the equation in slope-intercept form.  Solving the equation $3x+5y=20$ for $y$ in terms of $x$ gives $y = -\\frac{3}{5}x +4$.  So, the $y$-intercept is $\\boxed{(0,4)}$.<|im_end|>\n'}

In [16]:
dataset_test[10]

{'text': '<|im_start|>user\nSolve  \\[\\frac{x-9}{x+1}=2\\]for $x$.<|im_end|>\n<|im_start|>assistant\nCross-multiplication gives  \\[x-9=2x+2.\\]Simplifying this expression tells us  \\[x=\\boxed{-11}.\\]<|im_end|>\n'}

In [17]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=new_model,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16= False,
    bf16= False,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    evaluation_strategy="steps",  # or "epoch"
    # report_to="wandb"
    # eval_steps=eval_steps  # add this line if evaluation_strategy is "steps"
)

# Initialize SFT Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset= dataset_train,
    eval_dataset= dataset_test,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


/home/sayandeep/anaconda3/envs/sayandeep/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sayandeep/anaconda3/envs/sayandeep/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/sayandeep/anaconda3/envs/sayandeep/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sayandeep/anaconda3/envs/sayandeep/lib/python3.10/site-packages/trl/trainer/s

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [18]:
# Define a function to log training loss periodically
def log_training_loss(trainer, log_interval=25):
    step = 0
    while step < len(trainer.state.log_history):
        log = trainer.state.log_history[step]
        if "loss" in log:
            print(f"Step {step * log_interval}\tTraining Loss: {log['loss']}")
        step += 1

# Train model
trainer.train()
log_training_loss(trainer)

Step,Training Loss,Validation Loss
25,1.000100,0.957154
50,1.046400,0.887851
75,0.787900,0.833492
100,0.892500,0.874082
125,0.674200,0.838618
150,0.900200,0.868944
175,0.769800,0.841009
200,0.941000,0.863616
225,0.792800,0.835680
250,0.983700,0.915091


/home/sayandeep/anaconda3/envs/sayandeep/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Step 0	Training Loss: 1.0001
Step 50	Training Loss: 1.0464
Step 100	Training Loss: 0.7879
Step 150	Training Loss: 0.8925
Step 200	Training Loss: 0.6742
Step 250	Training Loss: 0.9002
Step 300	Training Loss: 0.7698
Step 350	Training Loss: 0.941
Step 400	Training Loss: 0.7928
Step 450	Training Loss: 0.9837
Step 500	Training Loss: 0.8216
Step 550	Training Loss: 0.974
Step 600	Training Loss: 0.7879
Step 650	Training Loss: 0.9312
Step 700	Training Loss: 0.8611
Step 750	Training Loss: 0.8751
Step 800	Training Loss: 0.7344
Step 850	Training Loss: 0.8773
Step 900	Training Loss: 0.7611
Step 950	Training Loss: 0.8804
Step 1000	Training Loss: 0.7938
Step 1050	Training Loss: 0.9367
Step 1100	Training Loss: 0.8283
Step 1150	Training Loss: 0.9533
Step 1200	Training Loss: 0.7635
Step 1250	Training Loss: 0.924
Step 1300	Training Loss: 0.8474
Step 1350	Training Loss: 0.8781
Step 1400	Training Loss: 0.7734
Step 1450	Training Loss: 0.9221
Step 1500	Training Loss: 0.756
Step 1550	Training Loss: 0.8568
Ste

In [19]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True

/home/sayandeep/anaconda3/envs/sayandeep/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


accuracy,▁
accuracy,0.95


In [20]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
trainer.model.push_to_hub(new_model, use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Sayandeep425/Llama-math-2-finetune/commit/3e8547533d6c5312556331d8e43642f370fe2956', commit_message='Upload model', commit_description='', oid='3e8547533d6c5312556331d8e43642f370fe2956', pr_url=None, pr_revision=None, pr_num=None)

In [21]:
messages = [{"role": "user", "content": "Find the $y$-intercept point of the line $3x+5y=20$. Provide your answer as an ordered pair."}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=250, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])


The $y$-intercept point is the point where the line crosses the $y$-axis.  The $x$-coordinate of this point is 0, so we have $y=\frac{20}{5}=\boxed{4}$.  The $y$-intercept point is $\boxed{(0,4)}$. [asy]
unitsize(0.5 cm);

pair A, B, C, D;

A = (0,4);
B = (8,0);
C = (8,4);
D = (0,0);

draw(A--B);
draw(C--D);
draw((-2,0)--(10,0));
draw((0,-2)--(0,6));
draw(A--C,red);
draw(A--D,red);

label("$y$", (0,6), N);
label("$x$", (10,0), E);
label("$y$-intercept", (0,4), S);
label("$x$-intercept", (


In [25]:
messages = [{"role": "user", "content": "Find the slope of the line passing through the points $(2,3)$ and $(5,7)$. Provide your answer as a simplified fraction."}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=300, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])


The slope of a line passing through two points is given by the formula $\frac{y_2-y_1}{x_2-x_1}$. Plugging in the given values, we get $\frac{7-3}{5-2}=\boxed{\frac{4}{3}}$.  (Note that this is the same as the slope of the line $y=\frac{4}{3}x-1$.) [asy]
size(150); defaultpen(linewidth(0.8));
pair A = (2,3), B = (5,7), C = (5,3), D = (7,7), E = (7,3), F = (3,7);
draw(A--B);
draw(C--D);
draw(E--F);
draw((-2,0)--(8,0), Arrows(4));
draw((0,-1)--(0,8), Arrows(4));
label("$y=\frac{4}{3}x-1$", (6,4), S);
label("$y=\frac{4}{3}x+1$", (6,5), N);
label("$y=-\frac{4}{3}x+7$", (7,6), E);



In [28]:
messages = [{"role": "user", "content": "What is the equation of the line that passes through the point $(4,-2)$ and has a slope of $3$? Provide your answer in slope-intercept form."}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=500, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])


The equation of the line is $y = mx + b$. We know that $m = 3$, so we plug that into the equation to get $y = 3x + b$. Since the line passes through $(4,-2)$, we plug those values into the equation to get $-2 = 3(4) + b \Rightarrow b = -16$. Therefore, the equation of the line is $\boxed{y = 3x - 16}$. [asy]
size(8cm);
real x = 4, y = -2;
real m = 3, b = -16;
real ticklen = 3;
real tickspace = 2;
real axisarrowsize = 0.14cm;
pen axispen = black + 0.7bp;
real vectorarrowsize = 0.2cm;
real tickdown = -0.5;
real tickdownlength = 0.15inch;
real tickdownbase = 0.3;
real wholetickdown = 0.2;
real wholetickdownlength = 0.2inch;
real wholetickdownbase = 0.4;
real wholetickdownbaseheight = 0.3;
real axisradius = 0.1;
real wholetickdownbaseheight = 0.3;
real axisradius = 0.1;
real axisarrowsize = 0.14cm;
real vectorarrowsize = 0.2cm;
real tickspace = 2;
real ticklen = 3;
real tickdown = -0.5;
real tickdownlength = 0.15inch;
real tickdownbase = 0.3;
real wholetickdown = 0.2;
real wholetickdownle

In [29]:
messages = [{"role": "user", "content": "Solve the quadratic equation $x^2 - 4x - 12 = 0$. Provide your answers as ordered pairs."}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=350, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])


We have $x^2 - 4x - 12 = 0$. Factoring, we get $(x-6)(x+2) = 0$, so either $x-6 = 0$ or $x+2 = 0$. The first equation gives us $x=6$, and the second gives us $x=-2$. Thus, our solutions are $x = \boxed{(6,-2)}$. (Note that this is an ordered pair, not an interval.) [asy]
size(150);
real ticklen = 3;
real tickspace = 2;
real axisarrowsize = 0.14cm;
pen axispen = black+1.2bp;
real vectorarrowsize = 0.2cm;
real tickdown = -0.5;
real tickdownlength = 0.15inch;
real tickdownbase = 0.3;
real wholetickdown = -0.75;
real wholetickdownlength = 0.15inch;
real wholetickdownbase = 0.3;
real wholetickdownbase2 = 0.2;
real wholetickdownlength2 = 0.2inch;
real wholetickdown2 = -0.5;
real wholetickdownlength3 = 0.2inch;
real wholetickdown3 = -0.5;
real wholetickdownbase3 = 0.2;
real wholetickdownlength4 = 0.


In [30]:
messages = [{"role": "user", "content": "Determine the equation of a circle with its center at $(3,-4)$ and radius $5$. Provide your answer in standard form"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=400, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])


The equation of a circle with center $(h,k)$ and radius $r$ is $(x-h)^2 + (y-k)^2 = r^2$.  So the equation of the circle is $(x-3)^2 + (y+4)^2 = 5^2$.  Simplifying, we get the equation of the circle: $(x-3)^2 + (y+4)^2 = 25$.  This can be rewritten as $x^2-6x+9+y^2+8y+16=25$, or $x^2-6x+y^2+8y-16=0$.  Therefore, the equation of the circle is $\boxed{x^2-6x+y^2+8y-16=0}$.  (We could also write this as $(x-3)^2+(y+4)^2=25$, but this is not in standard form.)  [asy]
size(150);
defaultpen(linewidth(0.7));
dotfactor=4;
xaxis(-6,6,Ticks(" ",1.0,begin=false,end=false,NoZero,Size=3));
yaxis(-6,6,Ticks(" ",1.0,begin=false,end=false,NoZero,Size=3));
real f(real x) {return 5*sqrt(1-(x-3)^2/25);}
draw(graph(f,-3,3));
dot((3,-4));
dot((3,-4)+5*dir(0));
dot((3,-4)+5*dir(90));
dot((3,-4)+5*dir(180));
dot((3,-4)+5*dir(270));
dot((3,-4)+5*dir(360));
[/


In [32]:
messages = [{"role": "user", "content": "If the function $f(x) = 2x^2 - 3x + 5$, find $f'(x)$. Provide your answer in the simplest form"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=400, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])


We have that
\[f'(x) = \frac{d}{dx} (2x^2 - 3x + 5) = \boxed{4x - 3}.\]We could also compute this by differentiating $2x^2 - 3x + 5$ term-by-term.  Alternatively, we can notice that
\[f'(x) = 2(2x) - 3 = \boxed{4x - 3}.\]In general, $f(x) = ax^2 + bx + c$ implies that $f'(x) = 2ax + b$.  This is because
\begin{align*}
f'(x) &= \frac{d}{dx} (ax^2 + bx + c) \\
&= \frac{d}{dx} (ax^2) + \frac{d}{dx} (bx) + \frac{d}{dx} (c) \\
&= a \cdot \frac{d}{dx} (x^2) + b \cdot \frac{d}{dx} (x) + 0 \\
&= 2ax + b.
\end{align*}Note that this is consistent with our previous result, since $f(x) = 2x^2 - 3x + 5$ implies that $f'(x) = 2(2x) - 3 = 4x - 3$.  In general, we can check that the derivative of $ax^2 + bx + c$ is $2ax + b$ by differentiating term-by-term.  For example, the derivative of $2x^2 - 3x +


In [33]:
messages = [{"role": "user", "content": "Find the area of a triangle with vertices at $(0,0)$, $(4,0)$, and $(4,3)$. Provide your answer in square units"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=400, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])



The base of the triangle has length $4$ units, and its height is $3$ units.  Therefore, the area of the triangle is $\frac{1}{2}(4)(3)=\boxed{6}$ square units. [asy]
size(150);
defaultpen(linewidth(0.7));
draw((0,0)--(4,0)--(4,3)--cycle);
draw((0,0)--(4,0),EndArrow);
draw((0,0)--(4,3),EndArrow);
draw((0,0)--(4,3),EndArrow);
label("4",(4,0),E);
label("3",(4,3),N);
label("0",(0,0),W);
[/asy]  OR

The area of a triangle is $\frac{1}{2}$ times the product of its base and its height.  Therefore, the area of the triangle is $\frac{1}{2}(4)(3)=\boxed{6}$ square units. [asy]
size(150);
defaultpen(linewidth(0.7));
draw((0,0)--(4,0)--(4,3)--cycle);
draw((0,0)--(4,0),EndArrow);
draw((0,0)--(4,3),EndArrow);
draw((0,0)--(4,3),EndArrow);
label("4",(4,0),E);
label("3",(4,3),N);
label("0",(0,0),W);
label("$\frac{1}{2}$",(2,1.5),NE);
label("$4$",(4,0),E);
label("$3$",(4,3


In [34]:
messages = [{"role": "user", "content": "Find the area of a triangle with vertices at $(0,0)$, $(4,0)$, and $(4,3)$. Provide your answer in square units"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=300, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])


The area of the triangle is equal to $\frac{1}{2} \cdot 4 \cdot 3 = \boxed{6}$ square units. [asy]
size(150);
defaultpen(linewidth(0.8));
draw((0,0)--(4,0)--(4,3)--cycle);
draw((0,0)--(4,3)--(4,0),dashed);
label("$(0,0)$",(0,0),S);
label("$(4,0)$",(4,0),S);
label("$(4,3)$",(4,3),NE);
[/asy] To see why this is true, draw a line connecting the third vertex to the midpoint of the segment connecting the first two vertices: [asy]
size(150);
defaultpen(linewidth(0.8));
draw((0,0)--(4,0)--(4,3)--cycle);
draw((0,0)--(4,3)--(4,0),dashed);
draw((2,0)--(4,3),dashed);
label("$(0,0)$",(0,0),S);
label("$(4,0)$",(4,0),S);
label("$(4


In [ ]:
messages = [{"role": "user", "content": "Find the $(x,y)$  intersection points of $y=4x^2$ and $y=4x+3$.Provide one worded answer in simplified form."}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=700, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])
#wrong answer

In [53]:
messages = [{"role": "user", "content": "Find the integral of $\\int_0^{\\frac{\\pi}{2}} \\frac{\\sin(x)}{\\sin(x) + \\cos(x)} dx$. Provide your answer in simplified form."}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=400, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])


We can write $\frac{\sin(x)}{\sin(x) + \cos(x)}$ as $\frac{\sin(x)}{1 + \cos(x)}$. We can then substitute $u = \cos(x)$, so $du = -\sin(x) dx$. We can then substitute and get $\int_0^{\frac{\pi}{2}} \frac{\sin(x)}{\sin(x) + \cos(x)} dx = \int_0^1 \frac{du}{1 + u} = \int_0^1 \frac{1 - u}{1 + u} du = \int_0^1 1 - u du = \boxed{\frac{\pi}{4}}$. Note that we can also solve this problem by using a substitution. We can substitute $u = \sin(x) + \cos(x)$, so $du = \cos(x) - \sin(x) dx$. We can then substitute and get $\int_0^{\frac{\pi}{2}} \frac{\sin(x)}{\sin(x) + \cos(x)} dx = \int_0^1 \frac{du}{u} = \boxed{\frac{\pi}{4}}$. We can also solve this problem by using a trigonometric substitution. We can substitute $u = \tan(\frac{x}{2})$, so $du = \frac{1}{2} \sec^2(\frac{x}{2}) dx$. We can then substitute and get $\int_0^{\frac{\pi}{2}} \frac{\sin(x)}{\sin(x) + \cos(x)} dx = \int_0^1 \frac{du}{1


In [ ]:
# Most efficient till now